<a href="https://colab.research.google.com/github/aharrisonau/gym-ReverseTruck/blob/main/ReverseTruck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ReverseTruck uses the ReverseTruck_env to simulate the reversing of a truck and trailer combination into a loading bay

The environment and the simulation have been developed by Andrew Harrison as a submission for SIT796 - Reinforcement Learning at Deakin University

First step is to install the environment for Github

In [1]:
# helper code from https://stackoverflow.com/questions/55835318/how-to-create-and-use-a-custom-openai-gym-environment-on-google-colab
%%capture
!rm -r ReverseTruck_env
!git clone https://github.com/aharrisonau/gym-ReverseTruck.git
!pip install -e gym-ReverseTruck




In [2]:
import gym_ReverseTruck

In [3]:
# preliminaries
import pandas as pd
import numpy as np
import math as math
import gym
from gym import error, spaces, utils
from gym.utils import seeding

Then we can create the environment

Note that the environment requires a truck starting position, a truck setup file and a dictionary of obstacles passed to it

In [4]:

# the initial truck model has a prime mover length of 5m and a trailer of 10m
# width of both is set as 2.5m, but not used yet
TruckDefinition = np.array( [5.0, 2.5, 10.0, 2.5])

# The truck starts with the back of the truck 20m out and 20m forward of the 
# end point (0,0) and perpendicular to the finish position
# Note that the position is consistent with the truck definition
StartPosition = np.array( [[20.0,35.0],
                         [20.0,30],
                         [20.0,20.0]])

# No obstacles are used yet
Obstacles = {}


In [5]:
env = gym.make('ReverseTruck-v0',TruckDefinition=TruckDefinition, StartPosition=StartPosition, Obstacles=Obstacles)

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Follow the method of Practical 1 to run the model

We need to define a policy to take actions.

The simplest policy is to make the back of the truck follow a 20m radius curve till it gets to the loading dock (x=0)

The geometry of this means that the trailer angle should be held at 0.253 rad to the direction of the prime mover.

The policy is then if the trailer angle is less than this, apply "alpha" right steering to increase the angle
If angle greater than this, apply "alpha" left steering to decrease the angle.

"alpha" is an adjustable parameter.  1 means full 45 deg steerign lock.  Lower it to prevent overcorrection if needed

In [27]:
def policy(obs, t):
    # Write the code for your policy here. You can use the observation
    # (a tuple of position and velocity), the current time step, or both,
    # if you want.
    pmX, pmY, pivX, pivY, trlX, trlY = obs
    alpha=0.2

    pmVect=np.array([pmX-pivX,pmY-pivY]) # prime mover vector
    trlVect=np.array([pivX-trlX,pivY-trlY]) # trailer vector
    pm_trlSin = np.cross(trlVect,pmVect,)/np.linalg.norm(pmVect)/np.linalg.norm(trlVect) #sine of the trailer angle relative to the PM
    pm_trlAngle=math.asin(np.clip(pm_trlSin,-1,1)) #relative trailer angle.  Clip eliminate rounding errors giving arguments >1    

    if pm_trlAngle<0.253:
      actions=[-1.0,-alpha]
    else:
      actions=[-1.0,alpha]
    
   
    return actions

In [29]:
o = env.reset()

for t in range(350):
    
    action = policy(o,t)  # Call your policy
    o, r, d, _ = env.step(action)  # Pass the action chosen by the policy to the environment
    if t%20==0:
      print(t,o,action)

    if d and t<TIME_LIMIT-1:
        print("Task completed in", t, "time steps")
        break
else:
    print("Time limit exceeded. Try again.")

env.reset()

0 [19.98416156 34.89997491 20.         29.9        20.         19.9       ] [-1.0, -0.2]
20 [19.73413559 32.8903809  20.06649822 27.90143962 19.99559856 17.90169096] [-1.0, -0.2]
40 [19.61184165 30.86898897 20.2593949  25.91109882 19.96308115 15.91548988] [-1.0, -0.2]
60 [19.6488769  28.84989411 20.57791608 23.9369636  19.87035931 13.96202684] [-1.0, 0.2]
80 [20.21875439 26.91661534 20.8977046  21.96292716 19.68717686 12.03646643] [-1.0, 0.2]
100 [20.69058852 24.95700445 21.11770606 19.97528079 19.4069972  10.12269356] [-1.0, 0.2]
120 [21.06317238 22.97604625 21.23736031 17.97908131 19.03042127  8.22565011] [-1.0, 0.2]
140 [21.33491967 20.97875619 21.25573062 15.97938332 18.55836362  6.35004216] [-1.0, 0.2]
160 [21.50513247 18.97026379 21.17276984 13.98132251 17.99205784  4.50065422] [-1.0, 0.2]
180 [21.57337364 16.9557273  20.988691   11.99003035 17.33298212  2.68219534] [-1.0, 0.2]
200 [21.53946792 14.94032051 20.70396688 10.01062092 16.58285539  0.89928512] [-1.0, 0.2]
220 [21.40412

array([20., 35., 20., 30., 20., 20.])